In [1]:
import numpy as np
from PIL import Image
np.random.seed(0) 

m = 10  
n = 784 
lambda_ = 1e-5  # regularization parameter

true_x = np.array(Image.open('./data/4.jpg'))
true_x = true_x.reshape(784)

x=true_x+np.random.randn(784)*(2.4e-3)

O = np.random.randn(n, 100, 784)
A = [O[i] for i in range(m)]
y = [A[i] @ true_x for i in range(m)]

'''
Constant error setting:C=3 or 5 and Diminishing errors setting:C=20 or 30 and No error setting:C=0
Gaussian vector  e=C*r or C*r**t
'''
r=np.random.normal(loc=0, scale=1e-3, size=100)

# PG Algorithms

In [2]:
import numpy as np
import time

class pg_algorithms_time:

    def soft_thresholding(self, x, threshold):
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def gradient_smooth_part(self, A, x, y):
        grad = np.zeros_like(x)
        n = len(y)
        for i in range(n):
            residual = A[i] @ x - y[i]
            grad += (A[i].T @ residual) / n
        return grad

    def proximal_gradient_descent(self, x, A, y, lambda_, true_x, r, max_duration, alpha, C): 

        start_time = time.time()  
        current_time = start_time 
        iterations = 0  

        x0 = x.copy()
        y0 = y.copy()

        while (current_time - start_time) < max_duration:  
            #y0+=C*(r**(iterations+1))
            y0 += C*r

            grad = self.gradient_smooth_part(A, x0, y0)
            x_new = self.soft_thresholding(x0 - alpha * grad, alpha * lambda_)
            x0 = x_new

            iterations += 1
   
            y0 = y.copy()
            current_time = time.time()  

        return x0, (np.linalg.norm(((x0-true_x)))**2)/len(y0)

# B-PG Algorithms

In [3]:
import numpy as np
import time

class bpg_algorithms_time:

    def gradient_of_smooth_part(self, x, y, A):
        grad = np.zeros_like(x)
        for i in range(len(y)):
            residual = A[i] @ x - y[i]
            grad += 2 * A[i].T @ residual
        return grad / len(y)

    def soft_thresholding(self, x, lambda_):
        return np.sign(x) * np.maximum(np.abs(x) - lambda_, 0)

    def block_proximal_gradient(self, x, A, y, lambda_, true_x, r, max_duration, learning_rate, C): 

        start_time = time.time()  
        current_time = start_time
        iterations = 0  

        x0 = x.copy()
        y0 = y.copy()


        while (current_time - start_time) < max_duration:  
            
            # y0+=C*(r**(iterations+1))
            y0 += C*r

            grad = self.gradient_of_smooth_part(x0, y0, A)
            
            x_new = x0 - learning_rate * grad
            
            x_new = self.soft_thresholding(x_new, lambda_ * learning_rate)
            x0 = x_new
            
            y0 = y.copy()
            iterations += 1 
            current_time = time.time()

        return x0,(np.linalg.norm(((x0-true_x)))**2)/len(y)

# SPG Algorithms

In [ ]:
import numpy as np
import time

class spg_algorithm_time:

    def prox_l1(self, v, lambda_):

        return np.sign(v) * np.maximum(np.abs(v) - lambda_, 0)

    def gradient_smooth_part(self, x, A, y, idx):

        return 2 * A[idx].T @ (A[idx] @ x - y[idx]) / len(y)

    def run_spg(self, x, A, y, lambda_, true_x, r, max_duration, step_size, C): 

        start_time = time.time()  
        current_time = start_time
        iterations = 0 

        x0 = x.copy()
        y0 = y.copy()

        while (current_time - start_time) < max_duration:  
            # y0+=C*(r**(iterations+1))
            y0 += C*r

            idx = np.random.randint(len(y))
            
            grad_smooth = self.gradient_smooth_part(x0, A, y0, idx)
            
            x_temp = x0 - step_size * grad_smooth
            
            x0 = self.prox_l1(x_temp, step_size * lambda_)
                                    
            y0 = y.copy()
            iterations += 1  
            current_time = time.time()

        return x0, (np.linalg.norm(((x0-true_x)))**2)/len(y)

# ADMN Algorithms

In [5]:
import numpy as np
import time
from scipy.sparse.linalg import cg
import math

class admm_algorithm_time:
    
    def soft_threshold(self, x, threshold):

        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def admm_solver(self, x, A, y, lambda_, true_x, r, max_duration, rho, C):

        start_time = time.time()  
        current_time = start_time
        iterations = 0  

        x0 = x.copy()
        y0 = y.copy()

        m, n = len(y0), len(x0)
        
        z = x.copy()
        u = np.zeros(n)


        AtA = sum([A[i].T @ A[i] for i in range(m)]) / m + rho * np.eye(n)
        b1 = sum([A[i].T @ y0[i] for i in range(m)]) / m

        while (current_time - start_time) < max_duration:  
            
            #y0+=C*(r**(iterations+1))
            y0+=C*r

            b = b1 + rho * (z - u)
            x0, _ = cg(AtA, b, x0, maxiter=10, rtol=(1e-4/(iterations/2+1)/math.pow(0.999,(iterations+1)/3e2)*0.75 + 1e-9))
        
            z = self.soft_threshold(x0 + u, lambda_ / rho)

            u = u + x0 - z

            y0 = y.copy()
            iterations += 1  
            current_time = time.time()
            
        return x0, (np.linalg.norm(((x0-true_x)))**2)/len(y0)

# PGRR Algorithms

In [6]:
import numpy as np
from typing import List, Tuple
import time

class pgrr_algorithm_time:
    def soft_thresholding(self, x: np.ndarray, threshold: float) -> np.ndarray:
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def compute_gradient(self, x: np.ndarray, A: List[np.ndarray], y: List[np.ndarray]) -> np.ndarray:
        n = len(y)
        gradient = np.zeros_like(x)
        for i in range(n):
            gradient += 2 * A[i].T @ (A[i] @ x - y[i])
        return gradient / n

    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, max_duration, gamma, C):
        start_time = time.time()  
        current_time = start_time
        iterations = 0  

        x = initial_x.copy()
        n = len(y)
        y0 = y.copy()

        while (current_time - start_time) < max_duration:  
            
            y0 += C*r
            # y0+=C*(r**(iterations+1))
            permuted_indices = np.random.permutation(n)
            
            for i in permuted_indices:
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                x = self.soft_thresholding(x - gamma * gradient, gamma * lambda_)
                
                
                iterations += 1  
                current_time = time.time()
                
            y0 = y.copy()

        return x, (np.linalg.norm(((x-true_x)))**2)/len(y)

# ours Algorithms

In [7]:
import numpy as np
import time

class ours_algorithms_time:

    def soft_thresholding(self, x, threshold):
        return np.sign(x) * np.maximum(np.abs(x) - threshold, 0)

    def PG_RR(self, initial_x, A, y, lambda_, true_x, r, max_duration, gamma, C, momentum):

        start_time = time.time()  
        current_time = start_time
        iterations = 0  

        x = initial_x.copy()
        n = len(y)
        velocity = np.zeros_like(x)
        y0 = y.copy()

        while (current_time - start_time) < max_duration:  
            # y0+=C*(r**(iterations+1))
            y0 += C*r

            permuted_indices = np.random.permutation(n)
            
            for i in permuted_indices:
                gradient = 2 * A[i].T @ (A[i] @ x - y0[i])
                velocity = momentum * velocity + gamma * gradient
                x = self.soft_thresholding(x - velocity, gamma * lambda_)
                
                iterations += 1  
                current_time = time.time()
            y0 = y.copy()

        return x, (np.linalg.norm(((x-true_x)))**2)/len(y)

In [9]:
# PG algorithm
pg_time=pg_algorithms_time()

# B-PG algorithm
bpg_time = bpg_algorithms_time()

# SPG algorithm
spg_time = spg_algorithm_time()

# ADMM algorithm
admm_time = admm_algorithm_time()

# PGRR algorithm
pgrr_time=pgrr_algorithm_time()

# ours algorithm
ours_time = ours_algorithms_time()

max_duration = 55
C = 0 # 3 5 20 30 

pg_x,pg_errors= pg_time.proximal_gradient_descent(x, A, y, lambda_, true_x, r, max_duration, 1.6e-7, C)
bpg_x,bpg_errors = bpg_time.block_proximal_gradient(x, A, y, lambda_, true_x, r, max_duration, 6e-8, C)
spg_x,spg_errors= spg_time.run_spg(x, A, y, lambda_, true_x, r, max_duration, 8.8e-8, C)
admm_x,admm_errors = admm_time.admm_solver(x, A, y, lambda_, true_x, r, max_duration, 7e5, C)
pg_rr_x,pg_rr_errors= pgrr_time.PG_RR(x, A, y, lambda_, true_x, r, max_duration, 6.5e-8, C)
ours_x,ours_errors= ours_time.PG_RR(x, A, y, lambda_, true_x, r, max_duration, 6.5e-8, C, 0.9)

In [ ]:
# np.save('', np.array([pg_errors,bpg_errors,spg_errors,admm_errors,pg_rr_errors,ours_errors]))